# Galaxy Neighbor Analysis — Full Pipeline Demo

**Part 1**: Load catalogs, find faint neighbors around bright galaxies  
**Part 2**: Filter environments, compute d1s, cache results, plot KDE grids

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from galaxy_neighbors import AnalysisConfig, GalaxyModel, run_neighbor_analysis
from galaxy_d1s import D1sConfig, compute_d1s, save_d1s, load_d1s, load_or_compute_d1s, plot_d1s_grid

plt.style.use('seaborn-v0_8-ticks')
plt.rcParams.update({'font.size': 16, 'xtick.top': True, 'ytick.right': True,
                     'xtick.direction': 'in', 'ytick.direction': 'in'})

## 1. Configuration

All magnitude limits live here. Change these and everything downstream updates.

In [ ]:
cfg = AnalysisConfig(
    bright_limits=[-22.0, -21.5, -21.0],
    faint_limits=[-17.4, -17.6, -17.8, -18.0, -18.2, -18.4, -18.6, -18.8],
    preselect_faint_limit=-17.4,
    redshift=10.145,
    survey_area_arcmin2=12.24,
)

d1s_cfg = D1sConfig(
    min_neighbors=2,   # drop environments with fewer than 2 faint neighbors
    n_bins=5,          # histogram bins for the neighbor count filter
    plot_d_max=8.0,    # x-axis limit on KDE plots [cMpc]
    bw_fid=0.18,       # KDE bandwidth - fiducial
    bw_stoc=0.11,      # KDE bandwidth - stochastic
)

print(f"Search box half-side: {cfg.search_box_mpc:.3f} Mpc")
print(f"Bright limits: {cfg.bright_limits}  ->  keys: {cfg.bright_names}")
print(f"Faint limits:  {cfg.faint_limits}  ->  keys: {cfg.faint_names}")

## 2. Part 1 — Run neighbor search

Skip this cell if you already have saved d1s (jump to Section 4).

In [ ]:
HALO_CATALOG  = '/lustre/astro/ivannik/21cmFAST_cache/d12b21e80b7885d62d31717c2c2d8421/1952/ffa852ccaa39d8f82951cc98ff798ab4/10.5000/HaloCatalog.h5'
MUV_FIDUCIAL  = '/lustre/astro/ivannik/catalog_fiducial_bigger_new_save.h5'
MUV_STOCH     = '/lustre/astro/ivannik/catalog_stoch_bigger_new3.h5'

results_fid, results_stoc = run_neighbor_analysis(
    fiducial_halo_path=HALO_CATALOG,
    fiducial_muv_path=MUV_FIDUCIAL,
    stochastic_halo_path=HALO_CATALOG,
    stochastic_muv_path=MUV_STOCH,
    config=cfg,
    muv_index=0,
)

## 3. Part 2 — Compute d1s and save to cache

`load_or_compute_d1s` will:
- Compute d1s and save the `.npz` if the file does not exist yet
- Load from the cache on every subsequent run (fast, no recomputation)

In [ ]:
CACHE_FID  = 'd1s_fiducial.npz'
CACHE_STOC = 'd1s_stochastic.npz'

d1s_fid = load_or_compute_d1s(
    path=CACHE_FID,
    results=results_fid,   # pass None here if the cache already exists
    cfg=cfg,
    d1s_cfg=d1s_cfg,
)

d1s_stoc = load_or_compute_d1s(
    path=CACHE_STOC,
    results=results_stoc,
    cfg=cfg,
    d1s_cfg=d1s_cfg,
)

## 4. Load from cache only (no recomputation)

Use this cell on subsequent runs, skipping Sections 2 and 3 entirely.

In [ ]:
# d1s_fid  = load_d1s('d1s_fiducial.npz',  cfg)
# d1s_stoc = load_d1s('d1s_stochastic.npz', cfg)

## 5. Inspect d1s

In [ ]:
for bkey in cfg.bright_names:
    for fkey in cfg.faint_names:
        arr = d1s_fid[bkey][fkey]
        print(f"[fid]  {bkey} | {fkey}: n={len(arr):4d}  "
              f"median d1={np.median(arr):.2f} cMpc  mean={np.mean(arr):.2f} cMpc")

## 6. Plot: KDE grids

One figure per bright magnitude threshold. Each panel shows one faint limit.

In [ ]:
OUTPUT_DIR = '/groups/astro/ivannik/projects/Neighbors/'

filenames = {
    'M21.5': 'd1s_change_Muvfaint_broader.pdf',
    'M21':   'd1s_change_Muvfaint_Muv0_m21.pdf',
    'M22':   'd1s_change_Muvfaint_Muv0_m22.pdf',
}

for bright_key in cfg.bright_names:
    fig = plot_d1s_grid(
        d1s_fid=d1s_fid,
        d1s_stoc=d1s_stoc,
        cfg=cfg,
        d1s_cfg=d1s_cfg,
        bright_key=bright_key,
        n_cols=2,
        redshift_label=10.5,
    )
    fname = filenames.get(bright_key, f'd1s_{bright_key}.pdf')
    fig.savefig(OUTPUT_DIR + fname, bbox_inches='tight')
    plt.show()
    print(f"Saved: {fname}")

## 7. Rerunning with different magnitude limits

Just create a new `AnalysisConfig`, set `force_recompute=True`, and re-run.

In [ ]:
# cfg_new = AnalysisConfig(
#     bright_limits=[-21.0, -20.0],
#     faint_limits=[-17.0, -17.5, -18.0, -19.0],
#     preselect_faint_limit=-17.0,
#     redshift=10.145,
# )
# d1s_fid_new = load_or_compute_d1s(
#     path='d1s_fiducial_new.npz',
#     results=results_fid,
#     cfg=cfg_new,
#     d1s_cfg=d1s_cfg,
#     force_recompute=True,
# )